# COVID19 Data Analysis



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from seaborn import heatmap
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.neural_network import MLPRegressor

In [ ]:
plt.style.use('ggplot')

In [ ]:
csv_file_url = 'https://drive.google.com/uc?export=view&id=11SJRu_ghaKQCmKTNP6avuSgOr_RqB7tx'
dataset = pd.read_csv(csv_file_url, sep=';', index_col=0)
dataset.index = pd.to_datetime(dataset.index, format='%d.%m.%Y')
dataset.head(10)

In [ ]:
for i, col in enumerate(dataset.columns.tolist()):
    plt.figure(figsize=(10, 6))
    x_axis = dataset.index.values
    y_axis = dataset[col].values
    plt.plot(x_axis, y_axis, label=col,marker='o')
    plt.title(col)
    plt.xlabel("Day")
    plt.ylabel("Number of people")
    plt.xticks(pd.date_range(dataset.index.values[0],
                             dataset.index.values[-1],
                             freq='W'),
               rotation=90)
    counter = 0
    for j, k in zip(x_axis, y_axis):
        counter += 1
        if not counter%7:
            plt.annotate(str(k), xy=(j, k))
    plt.savefig("outputs\{}.png".format(col))

In [ ]:
# ilk 8 günün verisi öngörülebilir olmadığı için öteleyelim
dataset_shifted = dataset.iloc[9:,:]
dataset_shifted.head()

In [ ]:
# About Data.
dataset_shifted.describe()

In [ ]:
scaler = MinMaxScaler()
dataset_scaled = pd.DataFrame(scaler.fit_transform(dataset_shifted),
                              columns=dataset_shifted.columns)

In [ ]:
corr = dataset_scaled.corr()
corr

In [ ]:
heatmap(corr,
        xticklabels=dataset_shifted.columns,
        yticklabels=dataset_shifted.columns)

num_case ile num_recovered girdi değişkeni, num_intube, num_yogun_bakim ise çıktı değişkeni olarak alınmıştır.

In [ ]:
X = dataset_scaled.iloc[:, [1,4]]
y = dataset_scaled.iloc[:,[2,3]]

In [ ]:
X

In [ ]:
lr = LinearRegression()
lr.fit(X.values, y.values)
y_predicted_lr = lr.predict(X.values)
print(mean_squared_error(y.values, y_predicted_lr))
print(r2_score(y.values, y_predicted_lr))

In [ ]:
%matplotlib inline
ax = y.plot(linestyle='-')
pd.DataFrame(y_predicted_lr,columns=y.columns).plot(ax=ax,linestyle='--')
ax.legend(['actual int. care','actual intubated','predicted int. care','predicted intubated'])

plt.plot(dataset_shifted.index.values, y.values, label='actual')
plt.plot(dataset_shifted.index.values, y_predicted_lr, label='predict')
plt.xticks(pd.date_range(dataset.index.values[0],
                         dataset.index.values[-1],
                         freq='W'),
           rotation=90)
plt.title('LR Model')
plt.xlabel('Day')
plt.ylabel('Number of deaths')
plt.legend()
#plt.savefig("outputs/lr.png")

In [ ]:
mlp = MLPRegressor(hidden_layer_sizes=(75),max_iter=1000)
mlp.fit(X.values, y.values)
y_predicted_mlp = mlp.predict(X.values)


In [ ]:
# Model evaluation
r2_mlp = r2_score(y.values, y_predicted_mlp)
print("mean_squared_error: ", mean_squared_error(y.values, y_predicted_mlp))
print("r2_score: ", r2_score(y.values, y_predicted_mlp))

In [ ]:
ax = y.plot(linestyle='-')
pd.DataFrame(y_predicted_mlp,columns=y.columns).plot(ax=ax,linestyle='--')
ax.legend(['actual int. care','actual intubated','predicted int. care','predicted intubated'])

In [ ]:
mlp.activation

In [ ]:
mlp.coefs_[0].round(2)